In [1]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
from datasets import load_dataset
import numpy as np
from transformers import TrainingArguments, Trainer
from torch.utils.data import DataLoader , Dataset
from LP import LabelPropagation , LabelSpreading

from simCSE import SimCSE

In [4]:
train_data = load_dataset("json" , data_files="data/prep_data/train.json")

Using custom data configuration default-3a9b72ef64992df2


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3a9b72ef64992df2/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
frc_train_data = train_data["train"].shard(num_shards=2 , index=0)
type(frc_train_data)
data = DataLoader(frc_train_data , batch_size=8)
next(iter(data))

{'tweets': ['@USER She should ask a few native Americans what their take on this is.',
  'Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT',
  '@USER @USER Obama wanted liberals &amp; illegals to move into red states',
  '@USER @USER Oh noes! Tough shit.',
  '@USER Buy more icecream!!!',
  '@USER @USER @USER It’s not my fault you support gun control',
  '@USER you are a lying corrupt traitor!!! Nobody wants to hear anymore of your lies!!! #DeepStateCorruption URL',
  '@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER I like my soda like I like my boarders with a lot of ICE.'],
 'labels': tensor([1, 0, 0, 1, 0

In [28]:
data = train_data["train"].train_test_split(train_size = 0.2 , test_size = 0.8)
data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 2648
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 10592
    })
})

In [21]:
model_ckpt = "princeton-nlp/sup-simcse-roberta-base"

In [22]:
embbedings = SimCSE(model_ckpt)

In [23]:
embds = embbedings.encode(data["train"]["tweets"])

100%|██████████| 42/42 [00:06<00:00,  6.90it/s]


In [24]:
LP = LabelSpreading(kernel='knn',n_jobs=-1, n_neighbors=20)   
LP.fit(embds.numpy(),data["train"]["labels"])

LabelSpreading(kernel='knn', n_jobs=-1, n_neighbors=20)

In [2]:
"""preprocessing olid_data"""
import json , csv , os
from datasets import Dataset
from collections import defaultdict

def load_data(path : str) :
    """load review data (json format)"""
    with open(path ,'r') as file :
        data = json.load(file)
    return data
        

def prepare_data(path : str):
    #load train,test data
    train_data = load_data(os.path.join(path ,"olid_train.json"))
    test_data = load_data(os.path.join(path ,"olid_test.json"))
    #create a dict to obtain previous two data to convert them into dataset format
    dictData = defaultdict(list)
    olied_dataset = {}
    for data , data_name in zip([train_data , test_data] , ["train" , "olid_test"]):
        dictData.clear()
        for t_value , label_value in zip(data["tweet"].values(),data["subtask_a"].values()) :
            dictData["tweets"].append(t_value)
            dictData["labels"].append(1 if label_value =="OFF" else 0)
            dictData["labels_name"].append(label_value)
        olied_dataset[data_name] =  Dataset.from_dict(dictData)
    
    return olied_dataset

In [3]:
Dict_data = prepare_data("data/")
Dict_data

{'train': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 13240
 }),
 'olid_test': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 3887
 })}

In [4]:
train_data_20 = Dict_data["train"].train_test_split(train_size=0.2 , test_size=0.8)
train_data_20

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 2648
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 10592
    })
})

In [33]:
embds = embbedings.encode(train_data_20["train"]["tweets"])

100%|██████████| 42/42 [00:05<00:00,  7.09it/s]


In [34]:
LP = LabelSpreading(kernel='knn',n_jobs=-1, n_neighbors=20)   
LP.fit(embds.numpy(),train_data_20["train"]["labels"])

LabelSpreading(kernel='knn', n_jobs=-1, n_neighbors=20)

In [93]:
import joblib
LP = joblib.load("LP_Graph.joblib")

In [94]:
LP

LabelSpreading(kernel='knn', n_jobs=-1, n_neighbors=20)

# Dataset
   - Train set = 13450
   - Test set = 3887
   
   


In [35]:
test_embds = embbedings.encode(train_data_20["test"]["tweets"])

100%|██████████| 166/166 [00:24<00:00,  6.68it/s]


In [36]:
preds = LP.predict(test_embds)

In [37]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(train_data_20["test"]["labels"], preds) * 100
print(f"Accuracy of Data-Labeling : {acc:0.2f}")

Accuracy of Data-Labeling : 72.56


In [51]:
dataset_new = Dataset.from_dict({"tweets" : train_data_20["test"]["tweets"] , "labels" : train_data_20["test"]["labels"]})

In [52]:
dataset_new

Dataset({
    features: ['tweets', 'labels'],
    num_rows: 10592
})

In [54]:
training_data = Dataset.from_dict({"tweets" : train_data_20["train"]["tweets"] + dataset_new["tweets"],
                                  "labels" :train_data_20["train"]["labels"] + dataset_new["labels"] })

{'tweets': '@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER We have the biggest balls of them all.... just saying.🤣🤣🤣🇺🇸MAGA URL',
 'labels': 0}

# Train data with Classifier

In [55]:
from transformers import AutoConfig , AutoTokenizer , AutoModelForSequenceClassification

In [56]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def init_model():
    return AutoModelForSequenceClassification.from_pretrained(model_ckpt , num_labels=2).to(device)

In [57]:
def tokenize(batch):
    return tokenizer(batch["tweets"] , padding=True , truncation=True)

olid_encoded = training_data.map(tokenize, batched=True , batch_size=None)
olid_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['tweets', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 13240
})

In [59]:
olid_encoded_splits = olid_encoded.train_test_split(train_size=0.8 , test_size=0.2)
olid_encoded_splits

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 10592
    })
    test: Dataset({
        features: ['tweets', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2648
    })
})

In [73]:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels , preds , average="weighted")
    precision = precision_score(labels , preds)
    recall = recall_score(labels , preds)
    acc = accuracy_score(labels , preds)
    return {"accuracy" : acc ,
            "precision" : precision,
            "recall" : recall,
            "f1" : f1}

In [63]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(olid_encoded_splits["train"]["input_ids"]) // batch_size
model_name = "DistilBertForSeqClassification"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs = 2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  weight_decay=0.001,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  report_to = None,
                                  log_level="error")

In [64]:
trainer = Trainer(model=init_model() , args=training_args,
                  compute_metrics=compute_metrics ,
                  train_dataset= olid_encoded_splits["train"],
                  eval_dataset=olid_encoded_splits["test"],
                  tokenizer = tokenizer)
trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.495000,0.437557,0.795695,0.795725
2,0.396300,0.429710,0.804381,0.803423


TrainOutput(global_step=332, training_loss=0.4450773291559105, metrics={'train_runtime': 178.2379, 'train_samples_per_second': 118.852, 'train_steps_per_second': 1.863, 'total_flos': 937223403916032.0, 'train_loss': 0.4450773291559105, 'epoch': 2.0})

In [65]:
def forward_pass_with_label(batch):
    inputs = {k:v.to(device) for k,v in batch.items() if k in tekonizer.model_input_names}
    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),reduction="none")
    return {"loss": loss.cpu().numpy(),"predicted_label": pred_label.cpu().numpy()}

In [68]:
test_data_olid = Dict_data["olid_test"]
test_data_olid

Dataset({
    features: ['tweets', 'labels', 'labels_name'],
    num_rows: 3887
})

In [69]:
test_data_olid = test_data_olid.map(tokenize , batched=True , batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [70]:
preds = trainer.predict(test_data_olid)

In [71]:
test_data_olid

Dataset({
    features: ['tweets', 'labels', 'labels_name', 'input_ids', 'attention_mask'],
    num_rows: 3887
})

In [72]:
preds.metrics

{'test_loss': 0.20889508724212646,
 'test_accuracy': 0.9225623874453306,
 'test_f1': 0.9249016907513963,
 'test_runtime': 4.1347,
 'test_samples_per_second': 940.101,
 'test_steps_per_second': 117.543}